# Tercile-Based Probability Forecasts
In this ipy notebook, the probability and observed category for tercile-based forecasts is calculated.

The ipynb is structured as follows:
1. Terciles computation using a three-year-out cross-validation approach.
2. Computation of the probability associated to each tercile-based category.
3. Estimation of the "observed*" category.
4. Saving of the results.

> Reanalysis data from MERRA2 is used for verification. 

In [1]:
import numpy as np
import os
import sys

sys.path.append(os.path.abspath('../../py'))
from probabilistic import terciles, cross_validation

In [3]:
def probab_tercile(data, data_tercile):
    """
    """

    n_year = data.shape[0]
    n_point = data.shape[1]
    n_ensamble = data.shape[2]

    result = np.full((3, n_year, n_point), np.nan)
    
    for point in range(n_point):
        data_point = data[:,point,:]
        data_tercile_point = data_tercile[:,point,:]
        result[0,:,point] = np.nansum(np.transpose(data_point) <= data_tercile_point[:,0], axis = 0)/n_ensamble
        result[1,:,point] = np.nansum((np.transpose(data_point) > data_tercile_point[:,0]) &
        (np.transpose(data_point) <= data_tercile_point[:,1]), axis = 0)/n_ensamble
        result[2,:,point] = np.nansum(np.transpose(data_point) > data_tercile_point[:,1], axis = 0)/n_ensamble
    
    return result

def tercile_obser(data, data_tercile):
    n_year = data.shape[0]
    n_point = data.shape[1]

    result = np.full((n_year, n_point), np.nan)
    for point in range(n_point):
        data_point = data[:,point]
        data_point_tercil = data_tercile[:,point,:]
        
        result[:,point][np.where((data_point <= data_point_tercil[:,0]))] = 1
        result[:,point][np.where(((data_point > data_point_tercil[:,0]) & 
                                  (data_point <= data_point_tercil[:,1])))] = 2
        result[:,point][np.where(((data_point > data_point_tercil[:,1])))] = 3
    
    return(result)

In [4]:
path_in = '../../outputs/timeseries'
path_out = '../../outputs/probabilistic_forecasts'

season = 'djf'  # 'djf' or 'jja'
regions = ['r1', 'r2', 'r3']
datasets = ['merra2', 'cfsv2', 'seas5']

result = {}

for region in regions:
    file_name = os.path.join(
            path_in,
            f'timeseries_{season}_{region}.npz'
        )
    data_region = np.load(file_name)
    
    for dataset in datasets:
        data_arr= data_region[dataset]

        data_terciles = cross_validation(data_arr, terciles, 2)

        if dataset == 'merra2':
            result[dataset] = tercile_obser(data_arr, data_terciles)
        else:
            result[dataset] = probab_tercile(data_arr, data_terciles)
    
    file_name_out = os.path.join(path_out, f'terciles_prob_and_reference_{season}_{region}.npz')
    np.savez(file_name_out, **result)